In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
%matplotlib inline

In [2]:
# Create list of headers
headers = ['GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code',
    'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
    'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
    'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code',
    'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources',
    'NumArticles', 'AvgTone','Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_Lat',
    'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code',
    'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_FullName', 'ActionGeo_CountryCode',
    'ActionGeo_ADM1Code', 'ActionGeo_Lat', 'ActionGeo_Long', 'ActionGeo_FeatureID','DATEADDED', 'SOURCEURL']

In [6]:
# 1. LOADING DATA FROM http://data.gdeltproject.org/events/index.html
# 1.1. Loading data for training set: April 2013 - December 2015 on local file training_set.csv

#Month = ['04', '05', '06', '07', '08', '09', '10', '11', '12']
FMonth = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
'''Days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31']'''
Month = ['04']
Days = ['01', '02']

training_set = pd.DataFrame()
# Firstly, loading data for 2013 year, starting from April
i = 2013
for j in Month:
    for k in Days:
        path = r'http://data.gdeltproject.org/events/' + str(i) + j + k + r'.export.CSV.zip'
        try:
            local_filename, header = urllib.request.urlretrieve(path)
        except urllib.request.HTTPError:
            break
        training_set = training_set.append(pd.read_csv(local_filename, sep = '\t', header = None, compression = 'zip'))
        
training_set.to_csv('./dataset/training_set.csv', index = False)

# Secondly, loading data for 2014 and 2015 years
'''for i in range(2014, 2016):
    for j in FMonth:
        for k in Days:
            path = r'http://data.gdeltproject.org/events/' + str(i) + j + k + r'.export.CSV.zip'
            try:
                local_filename, header = urllib.request.urlretrieve(path)
            except urllib.request.HTTPError:
                break
            training_set = training_set.append(pd.read_csv(local_filename, sep = '\t', header = None, compression = 'zip'))
        
training_set.to_csv('./dataset/training_set.csv', index = False)'''


C:\Users\Сергей\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


"for i in range(2014, 2016):\n    for j in FMonth:\n        for k in Days:\n            path = r'http://data.gdeltproject.org/events/' + str(i) + j + k + r'.export.CSV.zip'\n            try:\n                local_filename, header = urllib.request.urlretrieve(path)\n            except urllib.request.HTTPError:\n                break\n            training_set = training_set.append(pd.read_csv(local_filename, sep = '\t', header = None, compression = 'zip'))\n        \ntraining_set.to_csv('./dataset/training_set.csv', index = False)"

In [7]:
# Checking data
training_set = pd.read_csv('./dataset/training_set.csv', header = 0, names = headers, index_col = 0, 
                          parse_dates = [1,56], infer_datetime_format = True)
training_set.head(3)

C:\Users\Сергей\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
GLOBALEVENTID,,,,,,,,,,,,,,,,,,,,,
253461011,2003-04-04,200304,2003,2003.2575,AFG,AFGHANISTAN,AFG,NaN,NaN,NaN,...,AG,1,Algeria,AG,AG,28.0000,3.00,AG,2013-04-01,BBC Monitoring
253461012,2003-04-04,200304,2003,2003.2575,AUS,AUSTRALIA,AUS,NaN,NaN,NaN,...,AS,1,Australia,AS,AS,-27.0000,133.00,AS,2013-04-01,http://www.bangkokpost.com/breakingnews/343522...
253461013,2003-04-04,200304,2003,2003.2575,BUS,SHOP OWNER,NaN,NaN,NaN,NaN,...,-1354145,4,"Tai Hang, Hong Kong (general), Hong Kong",HK,HK00,22.4667,114.15,-1354145,2013-04-01,http://www.bloomberg.com/news/2013-04-01/hong-...


In [3]:
# 1.2. Loading data for testing set: January 2016 - present on local file testing_set.csv

#Month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
'''Days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31']'''
Month = ['01']
Days = ['01', '02']

testing_set = pd.DataFrame()
i = 2016
for j in Month:
    for k in Days:
        path = r'http://data.gdeltproject.org/events/' + str(i) + j + k + r'.export.CSV.zip'
        try:
            local_filename, header = urllib.request.urlretrieve(path)
        except urllib.request.HTTPError:
            break
        testing_set = testing_set.append(pd.read_csv(local_filename, sep = '\t', header = None, compression = 'zip'))
        
testing_set.to_csv('./dataset/testing_set.csv', index = False)


C:\Users\Сергей\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (8,10,11,13,14,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Сергей\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Checking data
testing_set = pd.read_csv('./dataset/testing_set.csv', header = 0, names = headers, index_col = 0, 
                          parse_dates = [1,56], infer_datetime_format = True)
testing_set.tail(3)

C:\Users\Сергей\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
GLOBALEVENTID,,,,,,,,,,,,,,,,,,,,,
498886653,2016-01-02,201601,2016,2016.0055,sco,SCOTTISH,NaN,NaN,sco,NaN,...,-2591190,4,"Buckingham Palace, Westminster, United Kingdom",UK,UKP5,51.50,-0.133333,-2591190,2016-01-02,http://slinkingtowardretirement.com/?p=238310
498886654,2016-01-02,201601,2016,2016.0055,sco,SCOTTISH,NaN,NaN,sco,NaN,...,-2591190,4,"Balmoral, Belfast, United Kingdom",UK,UKR3,54.55,-5.966670,-2589140,2016-01-02,http://slinkingtowardretirement.com/?p=238310
498886655,2016-01-02,201601,2016,2016.0055,sco,SCOTTISH,NaN,NaN,sco,NaN,...,-2591190,4,"Buckingham Palace, Westminster, United Kingdom",UK,UKP5,51.50,-0.133333,-2591190,2016-01-02,http://slinkingtowardretirement.com/?p=238310


In [3]:
# drawning a map
from mpl_toolkits.basemap import Basemap


UnicodeEncodeError: 'ascii' codec can't encode characters in position 9-14: ordinal not in range(128)